In [1]:
import xml.etree.ElementTree as et
import os, re
import shutil as sh
import pandas as pd
import time
from multiprocessing import Pool, Queue, Manager
import multiprocessing
import re
from functools import reduce
import numpy as np
from sqlalchemy import create_engine

# First:

I created a list of all possible directories starting from the MLB directory and created EVERY possible subdirectory under it.

In [2]:
def list_of_paths():
    
    dir_path_list = []
    for root, direct, file in os.walk("MLB", topdown=False):
        for name in file:
            dir_path_list.append(os.path.join(root,name))
            
    return dir_path_list

# gid_date function:

will take a subdirectory list index from list of paths function and extract the YEAR MONTH DAY format utilizing regex pattern recognition

In [3]:
def gid_date(file_name):
    
    gid = file_name
    r = '\d{4}_\d{2}_\d{2}'
    m = re.search(r, gid)
    date = m.group().split('_')
    
    return date

# game_name function:
Is pretty straightforward. game name takes the subdirectorry path and extracts just the gid name

In [4]:
def game_name(file_name):
    
    gid = file_name
    r = "([g]\S+\d)"
    m = re.search(r,gid)
    game_name = m.group()
    
    return game_name

This apparently is a python code that merges the first element (assuming its a dictionary) and merge it into the second dictionary. This is what allows me to carry parent tree information all the way down from the Game element and walk that dictionary down to every individual node/leaf

In [5]:
def merge_attrib(element,parent_element):
    
    merged_dict = {**parent_element,**element.attrib}
    
    return merged_dict

# Inning Parser

### Fairly simple code
The iterate command searches for the inning elements and iterates through them and carries every parent element down to every action whether it be atbat bottom so on etc. I then get to those elements and append them into empty list. This entire function then returns the relevant organized information for the entire inning as a list of list full of dictionaries

In [6]:
def InningParser(inning_file):
    tree = et.parse(inning_file)
    root = tree.getroot()
    name = game_name(inning_file)

    atbat_info = []
    action_info = []
    pitch_info = []
    pickoff_info = []
    runner_info = []

    for inning in root.iter('inning'):
        inn_dict = inning.attrib
        inn_dict['inning'] = inning.attrib['num']
        inn_dict['Game'] = name

        for sub_ele in inning:
            if sub_ele.tag =='top':
                inn_dict['Half'] = sub_ele.tag
            elif sub_ele.tag == 'bottom':
                inn_dict['Half'] = sub_ele.tag

            for atbat_or_action in sub_ele:
                if atbat_or_action.tag == "atbat":
                    atbact_obj = merge_attrib(atbat_or_action,inn_dict)
                    atbat_info.append(atbact_obj)

                elif atbat_or_action.tag == "action":
                    action_obj = merge_attrib(atbat_or_action,inn_dict)
                    action_info.append(action_obj)

                for pitch_po_runner in atbat_or_action:
                    if pitch_po_runner.tag == 'pitch':
                        pitch_obj = merge_attrib(pitch_po_runner,atbact_obj)
                        pitch_info.append(pitch_obj)
                    elif pitch_po_runner.tag == 'po':
                        po_obj = merge_attrib(pitch_po_runner,atbact_obj)
                        pickoff_info.append(po_obj)
                    elif pitch_po_runner.tag == 'runner':
                        run_obj = merge_attrib(pitch_po_runner,atbact_obj)
                        runner_info.append(run_obj)

    return [atbat_info, action_info, pitch_info, pickoff_info, runner_info]

# GameParser
performs the same basic function as the InningParser however for a relevant player xml file and returns again a list of list of dicitonaries

In [7]:
def GameParser(player_file):
    tree = et.parse(player_file)
    root = tree.getroot()
    name = game_name(player_file)

    team_info = []
    umpire_info = []
    player_info = []
    coach_info = []

    for element in root.iter('team'):
        team_dict = element.attrib
        team_dict['Game'] = name
        team_dict['ID'] = element.attrib['id']
        team_info.append(team_dict)
        del team_dict['id']

        for sub_ele in element:
                if sub_ele.tag == 'player':
                    player_obj = merge_attrib(sub_ele,team_dict)
                    player_info.append(player_obj)
                elif sub_ele.tag == 'coach':
                    coach_obj = merge_attrib(sub_ele,team_dict)
                    coach_info.append(coach_obj)

    for element in root.iter('umpire'):
        umpire_obj = merge_attrib(element,team_dict)
        umpire_info.append(umpire_obj)
        
    return [team_info,umpire_info,player_info,coach_info]

# The next few functions
are just automatiing the process of cleaning the information from the Inning and Game parser and preparing them for the SQL format we want them in for the table

In [8]:
def makeGamesTable(file,team_df):
    game = {}

    game_id = game_name(file)
    away_team = team_df[team_df['type']=='away']['ID'].values[0]
    home_team = team_df[team_df['type']=='home']['ID'].values[0]
    year = gid_date(file)[0]
    month = gid_date(file)[1]
    day = gid_date(file)[2]

    game['ID']=game_id
    game['Year']=year
    game['Month']=month
    game['Day']=day
    game['HomeTeam']=home_team
    game['AwayTeam']=away_team
    return game

In [9]:
def makeInningTable(df):
    inning_col = ['Game','inning','Half','num','event']
    adj_df = df[inning_col]
    adj_df.columns = ['Game','Number','Half','EventNum','EventType']
    return adj_df

In [10]:
def makeActionTable(df):
    df=df.rename(columns = {'num':'EventNum'})
    return df

In [11]:
def makeAtBatsTable(df):
    df=df.rename(columns = {'num':'EventNum'})
    return df

In [12]:
def makePitchesTable(df):
    df = df.rename(columns={'batter':'AtBatNum'})
    return df

In [13]:
def makePickoffsTable(df):
    df = df.rename(columns={'batter':'AtBatNum'})
    return df

In [14]:
def makeRunnersTable(df):
    df = df.rename(columns={'batter':'AtBatNum'})
    return df

In [15]:
def makeTeamsTable(df):
    df = df[['ID','name']]
    return df

In [16]:
def makePlayersTable(df):
    return df

In [17]:
def makeCoachesTable(df):
    df = df.rename(columns={'ID':'TeamID'})
    return df

In [18]:
def makeUmpireTables(df):
    return df

# MEAT and POTATOES

### MLB Aggregator Function

This is the function that does most of the heavy lifting. It cleans sorts and performs all of the above action from one simple file input. A thing of beauty really.

In [27]:
def MLBaggregator(file):
    if 'inning_all' in file:
        inning_file = file
        inning_info = InningParser(file)
        
        inning_info = InningParser(my_paths[5])
        atbat_df = makeInningTable(pd.DataFrame(inning_info[0]))
        action_df = makeInningTable(pd.DataFrame(inning_info[1]))
        pitch_df = makeInningTable(pd.DataFrame(inning_info[2]))
        pickoff_df = makeInningTable(pd.DataFrame(inning_info[3]))
        runner_df = makeInningTable(pd.DataFrame(inning_info[4]))

        inning_dfs = [atbat_df,action_df,pitch_df,pickoff_df,runner_df]
        Innings_Table = pd.concat([atbat_df,action_df,pitch_df,pickoff_df,runner_df])

        Actions_Table = makeActionTable(action_df)
        AtBats_Table = makeAtBatsTable(atbat_df)
        Pitches_Table = makePitchesTable(pitch_df)
        Pickoffs_Table = makePickoffsTable(pickoff_df)
        Runners_Table = makeRunnersTable(runner_df)
        
        return [Innings_Table,Actions_Table,AtBats_Table,Pitches_Table,Pickoffs_Table,Runners_Table]
        
    elif 'player' in file:
        player_info = GameParser(file)
        player_file = file
        
        team_df = pd.DataFrame(player_info[0])
        umpire_df = pd.DataFrame(player_info[1])
        player_df = pd.DataFrame(player_info[2])
        coach_df = pd.DataFrame(player_info[3])
        
        Games_Table = pd.DataFrame(columns=['ID','Year','Month','Day','HomeTeam','AwayTeam'])
        Games_Table = Games_Table.append(makeGamesTable(player_file,team_df),ignore_index=True)
        Teams_Table = makeTeamsTable(team_df)
        Players_Table = makePlayersTable(player_df)
        Coaches_Table = makeCoachesTable(coach_df)
        Umpires_Table = makeUmpireTables(umpire_df)
        
        return [Games_Table,Teams_Table,Players_Table,Coaches_Table,Umpires_Table]

Physically creating the list of paths and the database

In [20]:
my_paths = list_of_paths()
mlb = create_engine('sqlite:///Baseball2.db')

More heavy lifting because who likes typing

In [21]:
def RunProject(queue):
    while queue.empty()==False:
        file = queue.get()
        if file is None:
            break
        elif "inning_all"  in  file:
            inn_obj = MLBaggregator(file)

            inn_obj[0].to_sql('Innings',con=mlb,if_exists='append')
            inn_obj[1].to_sql('Actions',con=mlb,if_exists='append')
            inn_obj[2].to_sql('AtBats',con=mlb,if_exists='append')        
            inn_obj[3].to_sql('Pitches',con=mlb,if_exists='append')        
            inn_obj[4].to_sql('Pickoffs',con=mlb,if_exists='append')
            inn_obj[5].to_sql('Runners',con=mlb,if_exists='append')

        elif "player" in file:
            play_obj = MLBaggregator(file)

            play_obj[0].to_sql('Games',con=mlb,if_exists='append')
            play_obj[1].to_sql('Teams',con=mlb,if_exists='append')
            play_obj[2].to_sql('Players',con=mlb,if_exists='append')
            play_obj[3].to_sql('Coaches',con=mlb,if_exists='append')
            play_obj[4].to_sql('Umpires',con=mlb,if_exists='append')

In [22]:
queue = Queue()

def make_queue(list_of_dir):
    for i in my_paths:
        queue.put(i)

Final Project in four lines

In [23]:
make_queue(my_paths)

pool = Pool(30,RunProject,(queue,))
pool.close()
pool.join()

Process ForkPoolWorker-1:
Traceback (most recent call last):
Process ForkPoolWorker-3:
  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/engine/base.py", line 1244, in _execute_context
    cursor, statement, parameters, context
Process ForkPoolWorker-5:
Traceback (most recent call last):
  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/engine/default.py", line 552, in do_execute
    cursor.execute(statement, parameters)
Process ForkPoolWorker-9:
  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/engine/base.py", line 1244, in _execute_context
    cursor, statement, parameters, context
Traceback (most recent call last):
sqlite3.OperationalError: table "Games" already exists
  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/engine/default.py", line 552, in do_execute
    cursor.execute(statement, parameters)
Traceback (most recent call last):

The above exception was the direct cause of the following exception:



  File "/home/erich/anaconda3/lib/python3.7/site-packages/pandas/io/sql.py", line 569, in _execute_create
    self.table.create()
  File "/home/erich/anaconda3/lib/python3.7/site-packages/pandas/io/sql.py", line 569, in _execute_create
    self.table.create()
Traceback (most recent call last):

The above exception was the direct cause of the following exception:

  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/engine/base.py", line 2033, in _run_visitor
    conn._run_visitor(visitorcallable, element, **kwargs)
  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/sql/schema.py", line 860, in create
    bind._run_visitor(ddl.SchemaGenerator, self, checkfirst=checkfirst)
Traceback (most recent call last):
  File "/home/erich/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/engine/base.py", line 1607, in _run_visitor
    visitorcallable(self.d

  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/util/compat.py", line 383, in raise_from_cause
    reraise(type(exception), exception, tb=exc_tb, cause=cause)
  File "/home/erich/anaconda3/lib/python3.7/site-packages/pandas/io/sql.py", line 585, in create
    self._execute_create()
  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/engine/base.py", line 1466, in _handle_dbapi_exception
    util.raise_from_cause(sqlalchemy_exception, exc_info)
  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/engine/base.py", line 1244, in _execute_context
    cursor, statement, parameters, context
  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/engine/base.py", line 1248, in _execute_context
    e, statement, parameters, cursor, context
  File "/home/erich/anaconda3/lib/python3.7/site-packages/pandas/io/sql.py", line 1173, in to_sql
    table.create()
  File "/home/erich/anaconda3/lib/python3.7/site-packages/pandas/io/sql.py

  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/engine/base.py", line 1607, in _run_visitor
    visitorcallable(self.dialect, self, **kwargs).traverse_single(element)
Traceback (most recent call last):
  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/engine/base.py", line 1248, in _execute_context
    e, statement, parameters, cursor, context
  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/sql/schema.py", line 860, in create
    bind._run_visitor(ddl.SchemaGenerator, self, checkfirst=checkfirst)
  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/engine/base.py", line 2033, in _run_visitor
    conn._run_visitor(visitorcallable, element, **kwargs)
  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/engine/base.py", line 1466, in _handle_dbapi_exception
    util.raise_from_cause(sqlalchemy_exception, exc_info)
  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/engine/base.py"


The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/engine/base.py", line 2033, in _run_visitor
    conn._run_visitor(visitorcallable, element, **kwargs)
  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/engine/base.py", line 1607, in _run_visitor
    visitorcallable(self.dialect, self, **kwargs).traverse_single(element)
  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/sql/visitors.py", line 131, in traverse_single
    return meth(obj, **kw)
  File "/home/erich/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/sql/ddl.py", line 826, in visit_table
    include_foreign_key_constraints,
  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/engine/base.py", line 988, in execute
    return meth(self, multipa

...

Traceback (most recent call last):
  File "/home/erich/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/erich/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/erich/anaconda3/lib/python3.7/multiprocessing/pool.py", line 105, in worker
    initializer(*initargs)
  File "<ipython-input-21-552f7ce14370>", line 21, in RunProject
    play_obj[2].to_sql('Players',con=mlb,if_exists='append')
  File "/home/erich/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py", line 2531, in to_sql
    dtype=dtype, method=method)
  File "/home/erich/anaconda3/lib/python3.7/site-packages/pandas/io/sql.py", line 460, in to_sql
    chunksize=chunksize, dtype=dtype, method=method)
  File "/home/erich/anaconda3/lib/python3.7/site-packages/pandas/io/sql.py", line 1173, in to_sql
    table.create()
  File "/home/erich/anaconda3/lib/python3.7/site-packages/pandas/io/sql.

Process ForkPoolWorker-17:
Traceback (most recent call last):
  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/engine/base.py", line 1244, in _execute_context
    cursor, statement, parameters, context
  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/engine/default.py", line 552, in do_execute
    cursor.execute(statement, parameters)
Process ForkPoolWorker-30:
sqlite3.OperationalError: duplicate column name: id

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/engine/base.py", line 1244, in _execute_context
    cursor, statement, parameters, context
  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/engine/default.py", line 552, in do_execute
    cursor.execute(statement, parameters)
  File "/home/erich/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _b

  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/engine/default.py", line 552, in do_execute
    cursor.execute(statement, parameters)
sqlalchemy.exc.OperationalError: (sqlite3.OperationalError) duplicate column name: id
[SQL: 
CREATE TABLE "Players" (
	"index" BIGINT, 
	"Game" TEXT, 
	"ID" TEXT, 
	avg TEXT, 
	bat_order TEXT, 
	bats TEXT, 
	boxname TEXT, 
	current_position TEXT, 
	era TEXT, 
	first TEXT, 
	game_position TEXT, 
	hr TEXT, 
	id TEXT, 
	last TEXT, 
	losses TEXT, 
	name TEXT, 
	num TEXT, 
	parent_team_abbrev TEXT, 
	parent_team_id TEXT, 
	position TEXT, 
	rbi TEXT, 
	rl TEXT, 
	status TEXT, 
	team_abbrev TEXT, 
	team_id TEXT, 
	type TEXT, 
	wins TEXT
)

]
(Background on this error at: http://sqlalche.me/e/e3q8)
Process ForkPoolWorker-28:
Traceback (most recent call last):
  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/engine/base.py", line 1244, in _execute_context
    cursor, statement, parameters, context
  File "/home/erich/anaco

  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/engine/base.py", line 1248, in _execute_context
    e, statement, parameters, cursor, context
  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/engine/base.py", line 1466, in _handle_dbapi_exception
    util.raise_from_cause(sqlalchemy_exception, exc_info)
  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/util/compat.py", line 383, in raise_from_cause
    reraise(type(exception), exception, tb=exc_tb, cause=cause)
  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/util/compat.py", line 128, in reraise
    raise value.with_traceback(tb)
  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/engine/base.py", line 1244, in _execute_context
    cursor, statement, parameters, context
  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/engine/default.py", line 552, in do_execute
    cursor.execute(statement, parameters)
sqlalchemy.exc.Ope

  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/sql/visitors.py", line 131, in traverse_single
    return meth(obj, **kw)
  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/sql/ddl.py", line 826, in visit_table
    include_foreign_key_constraints,
  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/engine/base.py", line 988, in execute
    return meth(self, multiparams, params)
  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/sql/ddl.py", line 72, in _execute_on_connection
    return connection._execute_ddl(self, multiparams, params)
  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/engine/base.py", line 1050, in _execute_ddl
    compiled,
  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/engine/base.py", line 1248, in _execute_context
    e, statement, parameters, cursor, context
  File "/home/erich/anaconda3/lib/python3.7/site-packages/sqlalchemy/engine/base.py", line 146